# HOPUS

HOPUS (**HO**using **P**ricing **U**tilitie**S**) contains a variety of routines used to predict real estate prices.

This notebook highlights what HOPUS can do, namely
- clean the raw data,
- train a variety of models for the prediction of real estate prices, and
- evaluate the performance of these models.

## Technical preliminaries

In [1]:
# We clone the HOPUS repository to have access to all its data and routines
!git clone https://github.com/aremondtiedrez/hopus.git
%cd hopus

Cloning into 'hopus'...
remote: Enumerating objects: 237, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 237 (delta 16), reused 18 (delta 6), pack-reused 204 (from 1)
Receiving objects: 100% (237/237), 749.58 KiB | 4.16 MiB/s, done.
Resolving deltas: 100% (125/125), done.
/content/hopus


In [2]:
# Import requisite modules from HOPUS
import evaluation
import preprocessing

## Data cleaning

In [3]:
hpi = preprocessing.home_price_index.load()
preprocessing.home_price_index.preprocess(hpi)

listings_data = preprocessing.property_listings.load()
listings_data = preprocessing.property_listings.preprocess(listings_data, hpi)

preprocessing.property_listings.drop_outliers(listings_data)
preprocessing.property_listings.drop_missing_key_features(listings_data)

In [4]:
price_rmse = evaluation.hpi_rmse(listings_data, target="price")
log_price_rmse = evaluation.hpi_rmse(listings_data, target="logPrice")
print(
    "When using the available home price index\n"
    "instead of the true home price index,\n"
    f"the price RMSE is ${price_rmse/1_000:.0f}k and \n"
    f"the log-price RMSE is {log_price_rmse:.3f}."
)

When using the available home price index
instead of the true home price index,
the price RMSE is $10k and 
the log-price RMSE is 0.021.


In [5]:
preprocessing.property_listings.group_columns(listings_data)

## Linear regression: training and evaluation

In [6]:
import numpy as np
import secrets

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [22]:
seed = secrets.randbits(32)

# Train-test split
train_features, test_features = train_test_split(listings_data["keyPredictionFeatures"], train_size=0.8, shuffle=True, random_state=seed)
train_target, test_target = train_test_split(listings_data[("target", "price")], train_size=0.8, shuffle=True, random_state=seed)

# Train model
model = LinearRegression()
model.fit(train_features, train_target)

# Evaluate model
train_predictions = model.predict(train_features)
test_predictions = model.predict(test_features)

train_rmse = np.sqrt(mean_squared_error(train_target, train_predictions))
test_rmse = np.sqrt(mean_squared_error(test_target, test_predictions))

# Report evaluations
print(f"Seed: {seed}")
print(f"Training error: ${train_rmse / 1_000:.0f}k")
print(f"Test error:     ${test_rmse / 1_000:.0f}k")

Seed: 1745208383
Training error: $149k
Test error:     $189k


## Linear regression: evaluation with cross-validation

In [9]:
import numpy as np
import secrets

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [80]:
n_splits = 100
seed = secrets.randbits(32)
data = listings_data
features_label = "keyPredictionFeatures"
target_label = ("target", "price")

# Extract the features and target
features = data[features_label]
target = data[target_label]

# Split the data (virtually, i.e. by splitting the indices)
fold_indices = list(
    KFold(n_splits=n_splits, shuffle=True, random_state=seed).split(data)
)

# Train and evaluate the models
models = []
squared_errors = np.zeros(shape=(n_splits, 2))
for fold_number, (train_indices, test_indices) in enumerate(fold_indices):

    # Training
    model = LinearRegression()
    models.append(model)
    model.fit(features.iloc[train_indices], target.iloc[train_indices])

    # Evaluation
    train_predictions = model.predict(features.iloc[train_indices])
    test_predictions = model.predict(features.iloc[test_indices])

    train_mse = mean_squared_error(target.iloc[train_indices], train_predictions)
    test_mse = mean_squared_error(target.iloc[test_indices], test_predictions)
    squared_errors[fold_number] = train_mse, test_mse

train_cv_rmse, test_cv_rmse = np.sqrt(squared_errors.mean(axis=0))

# Report evaluations
print(f"Cross-validation training error: ${train_cv_rmse / 1_000:.0f}k")
print(f"Cross-validation test error:     ${test_cv_rmse / 1_000:.0f}k")

Cross-validation training error: $156k
Cross-validation test error:     $160k
